# VSM模型简介

在高中就学过，如果我们有向量$\vec{V_1}$和$\vec{V_2}$的话，有一个值可以表示两个向量的夹角余弦：

$$
\cos <\vec{V_1},\vec{V_2}>=\frac{\vec{V_1}\cdot\vec{V_2}}{|\vec{V_1}|\cdot|\vec{V_2}|}\tag{1}
$$

如果假设这两个向量是三维的：
$$
\begin{aligned}
\vec{V_1}=[x_1,y_1,z_1]\\
\vec{V_2}=[x_2,y_2,z_2]
\end{aligned}\tag{2}
$$

那么(1)可以写成：
$$
\cos <\vec{V_1},\vec{V_2}>=\frac{x_1 x_2+y_1 y_2+z_1 z_2}{\sqrt{x_1^2+y_1^2+z_1^2}\cdot\sqrt{x_2^2+y_2^2+z_2^2}}\tag{3}
$$

如果说这个向量不是三维的，而是$n$维的，那么假设：
$$
\begin{aligned}
V_1=[a_1,a_2,a_3\dots,a_n]\\
V_2=[b_1,b_2,b_3\dots,b_n]
\end{aligned}\tag{4}
$$

仍然可以定义广义的夹角余弦：
$$
\cos <\vec{V_1},\vec{V_2}>=\frac{\sum_{i=1}^n a_i b_i}{\sqrt{\sum_{i=1}^n a_i^2}\cdot\sqrt{\sum_{i=1}^n b_i^2}}\tag{5}
$$

容易想象的是，两个方向指向越近的向量，其夹角$<\vec{V_1},\vec{V_2}>$越小,而其余弦值$\cos <\vec{V_1},\vec{V_2}>$越大。

而两个向量指的方向越相似，则余弦值越大————于是根据这个思想可以构建一个向量空间模型**Vector Space Model**(VSM模型)，其在文本语义处理上有着简单直观的作用。具体方式如下流程：

- 现在手里有一本字典，里面有世界上所有词语，假设这个词语个数是$n$；同时每个词语都有一个标号$i$，$1\leq i\leq n$
- 随便说一句话，比如“今天天气如何”，然后把这句话分词得到“今天”，“天气”，“如何”三个词语
- 在字典中找到“今天”，“天气”，“如何”对应的序号$i,j,k$，并在一个$n$维数组中，把$i,j,k$位置令为1，其余位置设置为0，得到第一个向量$\vec{V_1}=[0,0\dots,1\text(“今天”对应位置),0\dots,1\text(“天气”对应位置),0\dots,1\text(“如何”对应位置),\dots]$
- 再说一句话“今天天气不错”，用同样的方法分词得到“今天、天气、不错”三个词，再类似地得到向量$\vec{V_2}$
- 用(5)求得$\cos <\vec{V_1},\vec{V_2}>$，这个值就表示**在当前文本空间**下，这两句话“今天天气如何”与“今天天气不错”的**相关程度**

当然在实际操作过程中，我们不可能取遍世间所有词汇，因此我们只需要取一篇文章/一个数据库中所有词段，让它“张成”一个字典，或说文本空间，然后在这个空间中讨论两个词段的夹角即可。

而且容易注意到，对于“这个问题我要再思考思考”这样的句子，思考出现了两次，因此在“思考”对应的位置上，可以置2，而不只是置1。

另外python的中文分词可以用jieba第三方库，这是个很好用的中文分词第三方库，尤其是jieba.lcut()函数可以把字符串分成中文分词。下面我们来看一个英语的例子：

现在我有一段话：

"Whenever you feel like criticizing any one, just remember that all the people in this world haven't had the advantages that you've had."

我引入了jieba库把它分词成为很多英语单词，如下打印出来：

In [50]:
text="Whenever you feel like criticizing any one, just remember that all the people in this world haven't had the advantages that you've had."
import jieba as jb
txt_list=jb.lcut(text)#这一步用python元组方法去重
txt_list=list(tuple(txt_list))
print(txt_list)

['Whenever', ' ', 'you', ' ', 'feel', ' ', 'like', ' ', 'criticizing', ' ', 'any', ' ', 'one', ',', ' ', 'just', ' ', 'remember', ' ', 'that', ' ', 'all', ' ', 'the', ' ', 'people', ' ', 'in', ' ', 'this', ' ', 'world', ' ', 'haven', "'", 't', ' ', 'had', ' ', 'the', ' ', 'advantages', ' ', 'that', ' ', 'you', "'", 've', ' ', 'had', '.']


上面这些词语就是所有词语，存放在txt_list的列表容器中，下面的讨论都假定**不超出**于这些词语的范畴。

为了方便起见，我用python的dict()字典容器给每个词语一个序号，这个字典叫做txt_dict。txt_dict\['词语'\]=词语对应的序号

In [51]:
txt_dict=dict()
for i in range(len(txt_list)):
    txt_dict[txt_list[i]]=i
    pass
print(txt_dict)
print('\n','advantages对应的词语序号是：',txt_dict['advantages'])

{'Whenever': 0, ' ': 48, 'you': 45, 'feel': 4, 'like': 6, 'criticizing': 8, 'any': 10, 'one': 12, ',': 13, 'just': 15, 'remember': 17, 'that': 43, 'all': 21, 'the': 39, 'people': 25, 'in': 27, 'this': 29, 'world': 31, 'haven': 33, "'": 46, 't': 35, 'had': 49, 'advantages': 41, 've': 47, '.': 50}

 advantages对应的词语序号是： 41


In [52]:
n=len(txt_list)#下属所有的向量都是n维的

现在假定有两条词语：

'You remember me.'

'People will remember you.'

第一句拆成'you','remember','me'三个词语。

第二句拆成'people','will','remember','you'三个词语。

对于不在字典txt_dict（或者txt_list）中的词语，我们不予考虑，剔除（比如'will'和'me'）,然后在对应单词词序上，把向量分量值由0置为1。

*注意，如有必要可以加一段大小写互换的代码，消除大小写不一致引起的索引问题*
*注意，空格也会算字符*

In [53]:
sentence1='you remember me'
sentence2='people will remember you'
list1=jb.lcut(sentence1)
list2=jb.lcut(sentence2)
#下面导入numpy库以进行向量运算
import numpy as np
V1=np.zeros(len(txt_list))#预设置n维度全零向量
V2=np.zeros(len(txt_list))
print(V1)#目前$向量V1全是0$

[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0.]


In [54]:
for item in list1:
    if item in txt_list:#这一步是为了筛选出在字典中的词语，对于字典以外的词语直接忽略不计
        k=txt_dict[item]#得到字符对应的 k 号词序
        V1[k]=1#把 k 号位置由0置1
        pass
    pass
V1

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 1., 0., 0.])

所以'you remember me'这句话在字典中张成的向量$\vec{V_1}$如上，类似的，'people will remember you'对应的$\vec{V_2}$如下：

In [55]:
for item in list2:
    if item in txt_list:#这一步是为了筛选出在字典中的词语，对于字典以外的词语直接忽略不计
        k=txt_dict[item]#得到字符对应的 k 号词序
        V2[k]=1#把 k 号位置由0置1
        pass
    pass
V2

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       1., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 1., 0., 0.])

为了求得$\cos <\vec{V_1},\vec{V_2}>$，我定义了以下函数：

In [56]:
def get_cos_v1v2(v1,v2):
    a=v1.dot(v2)#分子
    b=np.sqrt(v1.dot(v1))*np.sqrt(v2.dot(v2))
    return a/b
    pass
v1=np.array([1,1,1])#用作测试
v2=np.array([1,1,0])
print(get_cos_v1v2(v1,v2))

0.8164965809277259


那么我们可以得到$\cos <\vec{V_1},\vec{V_2}>$如下：

In [57]:
cos=get_cos_v1v2(V1,V2)
cos

0.8660254037844387

当然这只是$\cos\theta$余弦值，我们根据逻辑可以判断：

词语一'you remember'

词语二'people remember you'

(已经剔除所有不在字典里的词语)

其实是有2/3的相似程度的(也就是66.667%)，那么我们如何转化呢？可以使用$\arccos$函数，得到两个词段实际的相似程度是：

$$
\text{similarity}=\frac{\frac{\pi}{2}-\theta}{\frac{\pi}{2}}=\frac{\pi-2\theta}{\pi}\tag{6}
$$

In [58]:
theta=np.arccos(cos)#得到两个向量夹角值
similarity=(np.pi-2*theta)/np.pi#相似程度转换函数，见式子(6)
print(r'\theta=',theta,'\n','similarity=',similarity*100,'%')

\theta= 0.5235987755982987 
 similarity= 66.66666666666667 %


如上，相似程度是没问题的，有2/3的相似性质。可以用VSM模型计算词段的相关性质

当然如果你要用这个方法做查询，我建议你可以这么做（数据库不太大的时候）：

- 遍历数据库，用jieba仿照上面做一个“字典”
- 把每一个数据库中的词段，用上面方法构建在当下“字典”中对应的“词段向量$i$”
- 把输入的查询量，按照上述方法做成一个待查向量$\vec{u}$
- 遍历所有此段，算出$\vec{u}$和所有词段向量的相似程度，算法同上
- 排序，打印输出完成

当然我感觉这个方法还是会存在问题的，比如数据量太大遍历的成本有点高，或者这个方法没有考虑词语顺序，或者词语内涵相关程度等等，不过这些都可以优化，这个再说。我只是提供了一种模型VSM以解决你所提出的问题。